In [319]:
import pandas as pd
import numpy as np
import geopandas as gpd
from src.google_maps_funcs import get_distance_matrix, get_distance_matrix_batched, get_coordinates, unpack_distance_matrix_dict

In [320]:
%reload_ext autoreload
%autoreload 2

In [321]:
df = pd.read_csv("data/admsenternavn_coordinates.csv", sep=";")

In [ ]:
df.columns = ["KommuneNr."] + list(df.columns[1:])

# Add row IDs for easier tracking
df["ID"] = range(len(df))

In [323]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs="EPSG:4326")

In [324]:
gdf.explore()

In [ ]:
# Initialize DataFrames for distances and times
distance_df = pd.DataFrame(np.zeros((len(df), len(df))), index=df["Kommunenavn"], columns=df["Kommunenavn"])
time_df = pd.DataFrame(np.zeros((len(df), len(df))), index=df["Kommunenavn"], columns=df["Kommunenavn"])

In [326]:
coordinate_list = df[["lat", "lng"]].values.tolist()

In [327]:
coordinate_list = [(lat, lng) for lat, lng in coordinate_list]

In [328]:
distance_matrix = get_distance_matrix_batched(coordinate_list, coordinate_list)

Processing origin 1 of 357: (70.9822909, 25.9702074)
Processing origin 11 of 357: (70.23964620000001, 22.3479396)
Processing origin 21 of 357: (69.6492047, 18.9553238)
Processing origin 31 of 357: (69.1305608, 18.6120503)
Processing origin 41 of 357: (68.6905857, 17.5426984)
Processing origin 51 of 357: (68.08475729999999, 15.6086559)
Processing origin 61 of 357: (67.00638719999999, 14.5772046)
Processing origin 71 of 357: (65.9879382, 12.2913878)
Processing origin 81 of 357: (68.1465068, 13.6096372)
Processing origin 91 of 357: (64.29766430000001, 10.5106221)
Processing origin 101 of 357: (63.583683, 10.760819)
Processing origin 111 of 357: (63.292781, 9.0826007)
Processing origin 121 of 357: (62.89271429999999, 7.681804699999999)
Processing origin 131 of 357: (62.5962346, 6.4459551)
Processing origin 141 of 357: (62.3003856, 7.2615269)
Processing origin 151 of 357: (62.100751, 5.5581169)
Processing origin 161 of 357: (61.4654725, 11.3414611)
Processing origin 171 of 357: (61.31483170

In [350]:
distance_df, time_df = unpack_distance_matrix_dict(distance_matrix)

In [351]:
def find_col_names(matrix_df, col_name="ID"):
    new_cols = []
    for col in matrix_df.columns:
        n = df[(df.lat == col[0]) & (df.lng == col[1])][col_name].values[0]
        new_cols.append(n)
    return new_cols

def find_row_names(matrix_df, col_name="ID"):
    new_rows = []
    for row in matrix_df.index:
        n = df[(df.lat == row[0]) & (df.lng == row[1])][col_name].values[0]
        new_rows.append(n)
    return new_rows


In [352]:
time_df.columns = find_col_names(time_df)
distance_df.columns = find_col_names(distance_df)

time_df.index = find_row_names(time_df)
distance_df.index = find_row_names(distance_df)

In [356]:
distance_df = distance_df[distance_df.columns.sort_values()]
time_df = time_df[time_df.columns.sort_values()]

In [ ]:
# Add missing values for the first row and column
time_df[0] = pd.Series(np.nan, index=time_df.index)
distance_df[0] = pd.Series(np.nan, index=distance_df.index)
time_df.loc[356] = np.nan
distance_df.loc[356] = np.nan

In [ ]:
# Match enum id to KommuneNr.
time_df.index = df["KommuneNr."]
distance_df.index = df["KommuneNr."]
time_df.columns = df["KommuneNr."]
distance_df.columns = df["KommuneNr."]

In [360]:
distance_df.to_csv("data/result/distance_matrix.csv", sep=";")
time_df.to_csv("data/result/time_matrix.csv", sep=";")